## SMP model result visualisation

This notebook helps in visualisation of smp model outputs.   
You need to specify the **log_dir** of the training job. Below cells will automatically load the model and test dataset from the config file maintained inside the log folder.

In [121]:
log_dir = "../outputs/2022-01-17/03-50-40/"

In [122]:
import os
import cv2
import numpy as np
import torch
import matplotlib.pyplot as plt

import albumentations as albu
from omegaconf import DictConfig, OmegaConf

import segmentation_models_pytorch as smp

from catalyst.utils import load_checkpoint, unpack_checkpoint
from torch.utils.data import DataLoader
from torch.utils.data import Dataset as BaseDataset
import warnings

In [128]:
warnings.filterwarnings(action = "ignore")

### helper function for data visualization

In [129]:

def visualize(**images):
    print(images.keys())
    """PLot images in one row."""
    n = len(images)
    plt.figure(figsize=(30, 5))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')).title())
        plt.imshow(image)
    plt.show()

### Data loader functions

In [131]:
class Dataset(BaseDataset):
    """Read images, apply augmentation and preprocessing transformations.

    Args:
        images_dir (str): path to images folder
        masks_dir (str): path to segmentation masks folder
        class_values (list): values of classes to extract from segmentation mask
        augmentation (albumentations.Compose): data transfromation pipeline
            (e.g. flip, scale, etc.)
        preprocessing (albumentations.Compose): data preprocessing
            (e.g. noralization, shape manipulation, etc.)

    """

    CLASSES = ["background", "coords"]

    def __init__(
        self,
        images_dir,
        masks_dir,
        classes=None,
        augmentation=None,
        preprocessing=None,
    ):
        print(os.getcwd())
        self.ids = os.listdir(images_dir)

        self.images_fps = [os.path.join(images_dir, image_id) for image_id in self.ids]
        self.masks_fps = [os.path.join(masks_dir, image_id) for image_id in self.ids]

        # convert str names to class values on masks
        self.class_values = [self.CLASSES.index(cls.lower()) for cls in classes]

        self.augmentation = augmentation
        self.preprocessing = preprocessing

    def __getitem__(self, i):

        # read data
        image = cv2.imread(self.images_fps[i])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(self.masks_fps[i], 0)

        # extract certain classes from mask (e.g. cars)
        masks = [(mask == v) for v in self.class_values]
        mask = np.stack(masks, axis=-1).astype("float")

        # apply augmentations
        if self.augmentation:
            sample = self.augmentation(image=image, mask=mask)
            image, mask = sample["image"], sample["mask"]

        # apply preprocessing
        if self.preprocessing:
            sample = self.preprocessing(image=image, mask=mask)
            image, mask = sample["image"], sample["mask"]

        return image, mask

    def __len__(self):
        return len(self.ids)


def get_loader(cfg, split_type: str = "train"):

    preprocessing_fn = smp.encoders.get_preprocessing_fn(
        cfg.model.ENCODER, cfg.model.ENCODER_WEIGHTS
    )
    x_dir, y_dir = (
        os.path.join(cfg.processing.DATA_PATH, split_type),
        os.path.join(cfg.processing.DATA_PATH, f"{split_type}annot"),
    )

    if split_type == "train":

        dataset = Dataset(
            x_dir,
            y_dir,
            augmentation=get_training_augmentation(),
            preprocessing=get_preprocessing(preprocessing_fn),
            classes=cfg.model.CLASSES,
        )
        loader = DataLoader(
            dataset,
            batch_size=cfg.processing.TRAIN_BATCH_SIZE,
            shuffle=True,
            num_workers=cfg.processing.TRAIN_WORKERS,
        )
        return loader

    elif split_type == "val" or split_type == "test":

        dataset = Dataset(
            x_dir,
            y_dir,
            augmentation=get_validation_augmentation(),
            preprocessing=get_preprocessing(preprocessing_fn),
            classes=cfg.model.CLASSES,
        )
        loader = DataLoader(
            dataset,
            batch_size=cfg.processing.VALID_BATCH_SIZE,
            shuffle=False,
            num_workers=cfg.processing.VALID_WORKERS,
        )

        return loader

    

def get_validation_augmentation():
    """Add paddings to make image shape divisible by 32"""
    test_transform = [albu.PadIfNeeded(320, 320)]
    return albu.Compose(test_transform)


def to_tensor(x, **kwargs):
    return x.transpose(2, 0, 1).astype("float32")


def get_preprocessing(preprocessing_fn):
    """Construct preprocessing transform

    Args:
        preprocessing_fn (callbale): data normalization function
            (can be specific for each pretrained neural network)
    Return:
        transform: albumentations.Compose

    """

    _transform = [
        albu.Lambda(image=preprocessing_fn),
        albu.Lambda(image=to_tensor, mask=to_tensor),
    ]
    return albu.Compose(_transform)

def get_line(mask):
    filtered = np.where(mask>0)
    lower_index = np.argmax(filtered[0])
    l_col =filtered[1][lower_index]
    #get column values
    l_row = filtered[0][lower_index]
    col_filter = np.min(np.where(filtered[1]==l_col))
    u_row = filtered[0][col_filter]
    return (l_col,u_row),(l_col,l_row)
    

### Model loading from config file

In [132]:

cfg = OmegaConf.load(os.path.join(log_dir, ".hydra/config.yaml"))


#load model
model = eval(
    f"smp.{cfg.model.ARCH}(encoder_name='{cfg.model.ENCODER}',encoder_weights='{cfg.model.ENCODER_WEIGHTS}',classes={len(cfg.model.CLASSES)},activation=('{cfg.model.ACTIVATION}'))"
)

#load checkpoint
checkpoint = load_checkpoint(
    path=os.path.join(log_dir, cfg.training.LOG_DIR, "best_full.pth")
)

#remove unwanted value pairs
unpack_checkpoint(
    checkpoint=checkpoint,
    model=model,
)

# load dataloader
test_dataloader = get_loader(cfg, "test")

/Users/aman.gupta/Documents/self/synap/application/notebook


### Infer from model

In [ ]:
DEVICE = "cpu"
for image,gt_mask in test_dataloader:
    # print(img.shape,label.shape)
    gt_mask = gt_mask.squeeze()
    
    x_tensor = image.to(DEVICE)
    pr_mask = model.predict(x_tensor)
    pr_mask = (pr_mask.squeeze().cpu().numpy().round())
    gt_mask = gt_mask.numpy()  
    
    (l_col,u_row),(l_col,l_row) = get_line(pr_mask)
    
    img_cpy = image.numpy().squeeze().transpose([1,2,0])
    cv2.line(pr_mask,(l_col,u_row),(l_col,l_row),2,2)
    visualize(
    image=img_cpy, 
    GT=gt_mask,
    Pred = pr_mask
    )
    